In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np

# Biến toàn cục lưu dữ liệu
data = None

# Giao diện đăng nhập
def login():
    username = username_entry.get()
    password = password_entry.get()

    if username == "admin" and password == "admin":  # Kiểm tra thông tin đăng nhập
        messagebox.showinfo("Thành công", "Đăng nhập thành công!")
        root.withdraw()  # Ẩn cửa sổ đăng nhập
        show_algorithm_selection()  # Hiển thị cửa sổ lựa chọn giải thuật
    else:
        messagebox.showerror("Lỗi", "Thông tin đăng nhập không đúng!")

# Hiển thị cửa sổ lựa chọn giải thuật
def show_algorithm_selection():
    global algorithm_window
    algorithm_window = tk.Toplevel(root)
    algorithm_window.title("Chọn Giải Thuật")
    algorithm_window.geometry("600x600")

    # Nút tải dữ liệu
    load_button = tk.Button(algorithm_window, text="Tải Dữ Liệu", command=load_file)
    load_button.pack(pady=10)

    # Lựa chọn giải thuật
    algorithm_label = tk.Label(algorithm_window, text="Chọn giải thuật:")
    algorithm_label.pack(pady=10)

    kmeans_button = tk.Button(algorithm_window, text="K-Means", command=load_kmeans_interface)
    kmeans_button.pack(pady=10)

    apriori_button = tk.Button(algorithm_window, text="Apriori", command=load_apriori_interface)
    apriori_button.pack(pady=10)

    naive_bayes_button = tk.Button(algorithm_window, text="Naive Bayes", command=load_naive_bayes_interface)
    naive_bayes_button.pack(pady=10)

    # Hiển thị kết quả
    global result_label
    result_label = tk.Label(algorithm_window, text="")
    result_label.pack(pady=10)

# Tải giao diện K-Means
def load_kmeans_interface():
    kmeans_window = tk.Toplevel(root)
    kmeans_window.title("K-Means")
    kmeans_window.geometry("400x300")

    k_label = tk.Label(kmeans_window, text="Nhập số lượng cụm (K):")
    k_label.pack(pady=10)
    k_entry = tk.Entry(kmeans_window)
    k_entry.pack(pady=10)

    run_button = tk.Button(kmeans_window, text="Chạy K-Means", command=lambda: run_kmeans(k_entry))
    run_button.pack(pady=20)

# Chạy K-Means
def run_kmeans(k_entry):
    global data
    try:
        k = int(k_entry.get())  # Số lượng cụm
        if data is None:
            messagebox.showerror("Lỗi", "Vui lòng tải dữ liệu trước!")
            return
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data.select_dtypes(include=[np.number]))
        kmeans = KMeans(n_clusters=k)
        data['Cluster'] = kmeans.fit_predict(scaled_data)
        result_label.config(text=f"Kết quả phân cụm: {data['Cluster'].head()}")
    except Exception as e:
        messagebox.showerror("Lỗi", f"Có lỗi xảy ra: {e}")

# Tải giao diện Apriori
def load_apriori_interface():
    apriori_window = tk.Toplevel(root)
    apriori_window.title("Apriori")
    apriori_window.geometry("400x300")

    support_label = tk.Label(apriori_window, text="Nhập giá trị min_support:")
    support_label.pack(pady=10)
    support_entry = tk.Entry(apriori_window)
    support_entry.pack(pady=10)

    run_button = tk.Button(apriori_window, text="Chạy Apriori", command=lambda: run_apriori(support_entry))
    run_button.pack(pady=20)

# Chạy Apriori
def run_apriori(support_entry):
    global data
    try:
        min_support = float(support_entry.get())  # Nhập giá trị min_support
        if data is None:
            messagebox.showerror("Lỗi", "Vui lòng tải dữ liệu trước!")
            return
        frequent_itemsets = apriori(data, min_support=min_support, use_colnames=True)
        rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
        result_label.config(text=f"Kết quả Apriori: {rules.head()}")
    except Exception as e:
        messagebox.showerror("Lỗi", f"Có lỗi xảy ra: {e}")

# Tải giao diện Naive Bayes
def load_naive_bayes_interface():
    naive_bayes_window = tk.Toplevel(root)
    naive_bayes_window.title("Naive Bayes")
    naive_bayes_window.geometry("400x400")

    target_label = tk.Label(naive_bayes_window, text="Nhập cột mục tiêu:")
    target_label.pack(pady=10)
    target_entry = tk.Entry(naive_bayes_window)
    target_entry.pack(pady=10)

    input_label = tk.Label(naive_bayes_window, text="Dữ liệu đầu vào (dạng col:value, col:value):")
    input_label.pack(pady=10)
    input_text = tk.Text(naive_bayes_window, height=5, width=50)
    input_text.pack(pady=10)

    run_button = tk.Button(naive_bayes_window, text="Chạy Naive Bayes", command=lambda: run_naive_bayes(target_entry, input_text))
    run_button.pack(pady=20)

# Chạy Naive Bayes
def run_naive_bayes(target_entry, input_text):
    global data
    try:
        target_column = target_entry.get()
        input_values = input_text.get("1.0", tk.END).strip()
        input_data = {k: int(v) for k, v in (pair.split(":") for pair in input_values.split(","))}
        result = apply_naive_bayes(data, target_column, input_data)
        result_label.config(text=f"Kết quả phân lớp: {result}")
    except Exception as e:
        messagebox.showerror("Lỗi", f"Có lỗi xảy ra: {e}")

# Naive Bayes implementation
def apply_naive_bayes(df, target_column, input_data):
    df = df[list(input_data.keys()) + [target_column]]
    target_values = df[target_column].unique()
    priors = {value: len(df[df[target_column] == value]) / len(df) for value in target_values}

    likelihoods = {}
    for col, value in input_data.items():
        likelihoods[col] = {}
        for target_value in target_values:
            subset = df[df[target_column] == target_value]
            count_value = len(subset[subset[col] == value])
            likelihoods[col][target_value] = count_value / len(subset) if len(subset) > 0 else 0

    posteriors = {}
    for target_value in target_values:
        posteriors[target_value] = priors[target_value]
        for col in input_data:
            posteriors[target_value] *= likelihoods[col].get(target_value, 1e-6)

    predicted_class = max(posteriors, key=posteriors.get)
    return predicted_class

# Tải dữ liệu từ file
def load_file():
    global data
    filepath = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if filepath:
        try:
            data = pd.read_csv(filepath)
            messagebox.showinfo("Thành công", "Tệp dữ liệu đã được tải thành công!")
        except Exception as e:
            messagebox.showerror("Lỗi", f"Không thể tải tệp: {e}")

# Giao diện đăng nhập chính
root = tk.Tk()
root.title("Đăng Nhập")
root.geometry("400x300")

username_label = tk.Label(root, text="Tên đăng nhập:")
username_label.pack(pady=10)
username_entry = tk.Entry(root)
username_entry.pack(pady=10)

password_label = tk.Label(root, text="Mật khẩu:")
password_label.pack(pady=10)
password_entry = tk.Entry(root, show="*")
password_entry.pack(pady=10)

login_button = tk.Button(root, text="Đăng Nhập", command=login)
login_button.pack(pady=20)

root.mainloop()


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil